In [1]:
import math
import numpy as np

# Method interfaces

In [13]:
class IterativeMethod:
    def compute(self, A: [[float]], b: [float], tolerance: float, max_iteration_count: int) -> ([float], int, bool):
        raise AssertionError("Not implemented yet")
    def __repr__(self):
        raise AssertionError("Not implemented yet")

In [ ]:
class ForwardMethod:
    def compute(self, A: [[float]], b: [float]) -> [float]:
        raise AssertionError("Not implemented yet")

In [21]:
import time
import matplotlib.pyplot as plt

class IterativeMethodResult:
    def __init__(self, method: IterativeMethod, cond: float, x: [float], iterations: int , ok: bool, ex_time: int):
        self.cond = math.log10(cond)
        self.method = method
        self.x = x
        self.iterations = iterations
        self.ok = ok
        self.ex_time = ex_time 
        
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        return "method:{}; cond:{}; time:{}; iterations:{}".format(self.method, self.cond, self.ex_time, self.iterations)
    
    
class ForwardMethodResult:
    def __init__(self, method: ForwardMethod, x: [float], error: float, time: int):
        self.method = method
        self.x = x
        self.error = error
        self.time = time
        
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        return "method: {}; error:{}; time:{}".format(self.method, self.error, self.time)

class MethodsTest:
    def __init__(self, matrix_count = 10, size = 25):
        self.matrix_array = MethodsTest._n_rand_matrix(matrix_count, size)
        self.b_array = MethodsTest._n_rand_b(matrix_count, size)
        self.matrix_count = matrix_count
        
        self.hilbetr = MethodsTest._hilbert_matrix(size)
        self.hilbetr_b = np.random.rand(size)
        
        self.diag = MethodsTest._diag_matrix(size)
        self.diag_b = np.random.rand(size)
        
        self.size = size
    
    @staticmethod
    def _rand_matrix(size: int) -> [[float]]:
        A = np.array(np.random.rand(size, size), dtype=np.float64)
        A = np.tril(A) + np.tril(A, -1).T
        return A.T * A
    
    @staticmethod
    def _triangle_matrix(size: int) -> [[float]]:
        A = np.array(np.random.rand(size, size), dtype=np.float64)
        return np.tril(A).T
    
    @staticmethod
    def _diag_matrix(size: int) -> [[float]]:
        A = [[0 for i in range(size)] for j in range(size)]
        for i in range(size):
            A[i][i] = 0.16
        return np.array(A)
    
    @staticmethod
    def _hilbert_matrix(size: int) -> [[float]]:
        ans = np.zeros((size, size), dtype=np.float64)
        for i in range(size):
            for j in range(size):
                ans[i, j] = 1.0 / (i + j + 1)
        return ans
    
    @staticmethod
    def _n_rand_matrix(n: int, size: int) -> [[[float]]]:
        a = []
        for i in range(n):
            a.append(MethodsTest._rand_matrix(size))
        a.sort(key=lambda x: np.linalg.cond(x))
        return a
    
    @staticmethod
    def _n_rand_b(n: int, size: int) -> [[float]]:
        b = []
        for i in range(n):
            b.append(np.random.rand(size))
        return b
    
    def test_iterative_method(self, method: IterativeMethod, max_iteration_count = 100_000, tolerance = 1e-7) -> [IterativeMethodResult]: 
        results = []
        for i in range(len(self.matrix_array)):
            start = time.time()
            x, iterations, ok = method.compute(self.matrix_array[i], self.b_array[i], tolerance, max_iteration_count)
            end = time.time() - start
            results.append(IterativeMethodResult(method, np.linalg.cond(self.matrix_array[i]), x, iterations, ok, end))
        return results
    
    def test_iterative_method_hilbert(self, method, max_iteration_count = 100_000, tolerance = 1e-7) -> [IterativeMethodResult]:
        results = []
        start = time.time()
        x, iterations, ok = method.compute(self.hilbetr, self.hilbetr_b, tolerance, max_iteration_count)
        end = time.time() - start
        results.append(IterativeMethodResult(method, np.linalg.cond(self.hilbetr), x, iterations, ok, end))
        return results
    
    def test_iterative_method_diag(self, method, max_iteration_count = 100_000, tolerance = 1e-7) -> [IterativeMethodResult]:
        results = []
        start = time.time()
        x, iterations, ok = method.compute(self.diag, self.diag_b, tolerance, max_iteration_count)
        end = time.time() - start
        results.append(IterativeMethodResult(method, np.linalg.cond(self.hilbetr), x, iterations, ok, end))
        return results
    
    
    
    @staticmethod
    def draw_iterative_method_time(results: [IterativeMethodResult]):
        results = list(filter(lambda x: x.ok, results))
        times = list(map(lambda x: x.iterations,results))
        cond = list(map(lambda x: x.cond, results))
        plt.scatter(cond, times, s = 1)
    
    
    
    def test_forward_method(self, method: ForwardMethod) -> [ForwardMethod]:
        results = []
        for i in range(len(self.matrix_array)):
            start = time.time()
            x = method.compute(self.matrix_array[i], self.b_array[i])
            end = time.time() - start
            results.append(ForwardMethodResult(method, x, np.linalg.norm(np.dot(self.matrix_array[i], x) - self.b_array[i]), end))
        return results
    
    def test_forward_method_hilbert(self, method) -> [ForwardMethodResult]:
        results = []
        start = time.time()
        x = method.compute(self.hilbetr, self.hilbetr_b)
        end = time.time() - start
        results.append(ForwardMethodResult(method, x, np.linalg.norm(np.dot(self.hilbetr, x) - self.hilbetr_b), end))
        return results
    
    def test_forward_method_diag(self, method) -> [ForwardMethodResult]:
        results = []
        start = time.time()
        x = method.compute(self.diag, self.diag_b)
        end = time.time() - start
        results.append(ForwardMethodResult(method, x, np.linalg.norm(np.dot(self.hilbetr, x) - self.hilbetr_b), end))
        return results
        

# method_data = MethodsTest().test_forward_method(GaussMethod())
# print(method_data)
# print('\n'.join(list(map(str, temp))))


In [23]:
tester = MethodsTest()


## Матрица Гильберта

In [23]:
# print(tester.test_iterative_method_hilbert(GradientMethod()))
# print(tester.test_iterative_method_hilbert(JacobiMethod()))
# print(tester.test_iterative_method_hilbert(SeidelMethod()))
# print(tester.test_iterative_method_hilbert(SeidelRelaxMethod()))
print(tester.test_forward_method_hilbert(GaussMethod()))

[method: Gauss; error:7.848562903736077e+18; time:0.006999969482421875]


## Диагональная матрица

In [22]:
print(tester.test_iterative_method_diag(GradientMethod()))
print(tester.test_iterative_method_diag(JacobiMethod()))
print(tester.test_iterative_method_diag(SeidelMethod()))
print(tester.test_iterative_method_diag(SeidelRelaxMethod()))
print(tester.test_forward_method_diag(GaussMethod()))


TypeError: 'GaussMethod' object is not callable

## Случайные матрицы


# Methods

In [3]:
class GaussMethod(ForwardMethod):
    @staticmethod
    def _find_max_row(A: [[float]], ind: int) -> int:
        cur_max = A[ind, ind]
        cur_ind = ind
        for i in range(ind, len(A)):
            cur_val = A[i, ind]
            if cur_max < cur_val:
                cur_max = cur_val
                cur_ind = i
        return cur_ind
    
    @staticmethod
    def normalize_line(A: [[float]], ind: int):
        a = A[ind, ind]
        for j in range(ind, len(A)):
            A[ind, j] = A[ind, j] / a
            
    @staticmethod
    def go_down(A: [[float]], b: [float], ind: int):
        for i in range(ind + 1, len(A)):
            main_val = A[i, ind]
            b[i] -= main_val * b[ind]
            for j in range(ind, len(A)):
                A[i, j] -= main_val * A[ind, j]
            
    
    def compute(self, A: [[float]], b: [float]) -> [float]:
        # make matrix diagonal
        for i in range(len(A)):
            # find row with max value in 'i' column -- main row
            j = GaussMethod._find_max_row(A, i)
            # swap main row and current
            A[[i, j]] = A[[j, i]]
            
            b[i] /= A[i, i]
            GaussMethod.normalize_line(A, i)
            GaussMethod.go_down(A, b, i)
        # get solution
        ans = np.zeros(len(A))
        for i in reversed(range(len(A))):
            ans[i] = b[i]
            for j in reversed(range(i + 1, len(A))):
                ans[i] -= A[i][j] * ans[j]
        return ans
    def __repr__(self):
        return "Gauss"

In [4]:
class GradientMethod(IterativeMethod):
    def compute (self, A, b, tol = 1e-7, max_iter = 100000):
        iters = 0
        (n, m) = np.shape(A)
        x = np.random.rand(m)
        r = b - np.dot(A, x)
        z = r
        while iters < max_iter and np.linalg.norm(np.dot(A, x) - b) > tol:
            prodAz = np.dot(A, z)
            rNormSquared = np.linalg.norm(r) ** 2 # just for performance
            
            a = rNormSquared / np.dot(prodAz, z)
            x += a * z
            r -= a * prodAz
            beta = np.linalg.norm(r) ** 2 / rNormSquared
            z = r + beta * z
            iters += 1
        return tuple([x, iters, iters != max_iter])
    def __repr__(self):
        return "Gradient"

NameError: name 'IterativeMethod' is not defined

In [5]:
# same lenght
def deltaVectorsNorm(x, y):
    res = 0.0
    for i in range (0, len(x)):
        res += (y[i] - x[i]) ** 2
    return math.sqrt(res)

In [6]:
# Ax = b
# epsiolon - accuracy parameter
# method -
#   0 - Jacobi
#   1 - Seidel (determinant(B) should be less 1)
#   2 - Seidel with relaxation
# wRelax - relaxation parameter

def templateJakobiAndSeidel(A, b, epsilon, maxIterations, method, wRelax):
    iterations = 0
    B = []
    d = []
    xn = []
    x = []
    
    for i in range (0, len(A)):
        d.append(b[i] / A[i][i])
        xn.append(d[i])
        x.append(d[i] + 1000)
        B.append([])
        for j in range (0, len(A[i])):
            B[i].append(-A[i][j] / A[i][i])
        B[i][i] = 0.0
    
    # detB = determinant(B)
    # eps = (1 - detB) * epsilon / detB
    eps = epsilon
    
    while ((deltaVectorsNorm(x, xn) > eps) and iterations < maxIterations):
        x = xn.copy()
        for i in range (0, len(B)):
            xn[i] = 0.0
            for j in range (0, len(B[i])):
                xn[i] += B[i][j] * (xn[j] if (j < i and method > 0) else x[j])
            xn[i] += d[i]
            if (method == 2):
                xn[i] = wRelax * xn[i] + (1 - wRelax) * x[i]
        iterations = iterations + 1
            
    
    return (xn, iterations, True)

In [15]:
class JacobiMethod(IterativeMethod):
    def compute (self, A, b, tol = 1e-7, max_iter = 100000):
        return templateJakobiAndSeidel(A, b, tol, max_iter, 0, 0.5)
    def __repr__(self):
        return "Jacobi"

In [16]:
class SeidelMethod(IterativeMethod):
    def compute (self, A, b, tol = 1e-7, max_iter = 100000):
        return templateJakobiAndSeidel(A, b, tol, max_iter, 1, 0.5)
    def __repr__(self):
        return "SeidelMethod"

In [17]:
class SeidelRelaxMethod(IterativeMethod):
    def compute (self, A, b, tol = 1e-7, max_iter = 100000):
        return templateJakobiAndSeidel(A, b, tol, max_iter, 2, 0.5)
    def __repr__(self):
        return "SeidelRelax"

In [18]:
def test_jakobi():
    A = [[10,1,-1],[1,10,-1],[-1,1,10]]
    b = [11,10,10]
    (ans, it, p) = JacobiMethod().compute(A, b)
    print (ans)
    print (it)

test_jakobi()

NameError: name 'math' is not defined

In [19]:
def test_seidel():
    A = [[10,1,-1],[1,10,-1],[-1,1,10]]
    b = [11,10,10]
    (ans, it, p) = SeidelMethod().compute(A, b)
    print (ans)
    print (it)

test_seidel()

NameError: name 'math' is not defined

In [12]:
def test_seidel_relax():
    A = [[10,1,-1],[1,10,-1],[-1,1,10]]
    b = [11,10,10]
    (ans, it, p) = SeidelRelaxMethod().compute(A, b)
    print (ans)
    print (it)

test_seidel_relax()

NameError: name 'SeidelRelaxMethod' is not defined